In [ ]:
import os
print(os.getcwd())


In [2]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

In [3]:

import numpy as np
np.version
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib as mpl
import anndata
#import bbknn
import os
from scipy import sparse

from matplotlib import rcParams

import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('default')  

sc.settings.set_figure_params(dpi=80)

sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
#sc.logging.print_versions()
sc.logging.print_header()

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
2024-11-25 10:24:25.109494: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-25 10:24:25.227620: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-25 10:24:25.230718: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'

scanpy==1.9.8 anndata==0.10.8 umap==0.5.3 numpy==1.23.5 scipy==1.12.0 pandas==1.5.3 scikit-learn==1.5.1 statsmodels==0.13.5 igraph==0.9.9 louvain==0.7.0 pynndescent==0.5.4


In [4]:
import  scipy.signal.signaltools

def _centered(arr, newsize):
    # Return the center newsize portion of the array.
    newsize = np.asarray(newsize)
    currsize = np.array(arr.shape)
    startind = (currsize - newsize) // 2
    endind = startind + newsize
    myslice = [slice(startind[k], endind[k]) for k in range(len(endind))]
    return arr[tuple(myslice)]

scipy.signal.signaltools._centered = _centered

In [5]:
import seaborn as sns

In [17]:
adata = sc.read('./SCLC_Public.h5ad')
adata

/DATA05/home/youwone/.local/lib/python3.9/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


AnnData object with n_obs × n_vars = 147137 × 22397
    obs: 'ngenes', 'libsize', 'mito_frac', 'RBP_frac', 'batch', 'donor_id', 'treatment', 'procedure', 'histo', 'cell_type_coarse', 'cell_type_fine', 'cell_type_general', 'clusters', 'cell_type_med', 'H_knn', 'development_stage_ontology_term_id', 'sex_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'assay_ontology_term_id', 'is_primary_data', 'tissue_ontology_term_id', 'disease_ontology_term_id', 'organism_ontology_term_id', 'cell_type_ontology_term_id', 'suspension_type', 'HTAN_Biospecimen_ID', 'HTAN_Participant_ID', 'tissue_type', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid'
    var: 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype', 'feature_length', 'feature_type'
    uns: 'citation', 'default_embedding', 'schema_reference', 'schema_version', 'title'
    obsm: 'X_pca', 'X_umap'

In [14]:
Groups_tab = pd.crosstab(columns=adata.obs['treatment'],  # Make a crosstab
                        index=adata.obs['donor_id'], margins=True)    
Groups_tab

treatment,Immunotherapy,Naive,Platinum Doublet,"Platinum Doublet,Immunotherapy","Platinum Doublet,Immunotherapy,TKI","Platinum Doublet,Immunotherapy,TMZ,Other chemotherapy","Platinum Doublet,Immunotherapy,TMZ,Other chemotherapy,VEGF inhibitor","Platinum Doublet,PARP inhibitor,TMZ","Platinum Doublet,TKI",All
donor_id,,,,,,,,,,
PleuralEffusion,0,0,0,0,0,0,2549,0,0,2549
RU255,0,0,2559,0,0,0,0,0,0,2559
RU426,0,2121,0,0,0,0,0,0,0,2121
RU653,0,316,0,0,0,0,0,0,0,316
RU661,0,4434,0,0,0,0,0,0,0,4434
RU666,0,0,0,0,0,0,0,0,1208,1208
RU675,0,6175,0,0,0,0,0,0,0,6175
RU676,0,3900,0,0,0,0,0,0,0,3900
RU681,0,0,1300,0,0,0,0,0,0,1300


In [13]:
Groups_tab = pd.crosstab(columns=adata.obs['histo'],  # Make a crosstab
                        index=adata.obs['donor_id'], margins=True)    
Groups_tab

histo,LUAD,SCLC,normal,All
donor_id,,,,
PleuralEffusion,0,2549,0,2549
RU255,2559,0,0,2559
RU426,0,2121,0,2121
RU653,316,0,0,316
RU661,4434,0,0,4434
RU666,1208,0,0,1208
RU675,3196,0,2979,6175
RU676,3900,0,0,3900
RU681,1300,0,0,1300


In [10]:
adata.obs['histo'].unique()

['SCLC', 'LUAD', 'normal']
Categories (3, object): ['LUAD', 'SCLC', 'normal']

In [18]:
cells_of_interest = adata.obs.loc[~adata.obs['donor_id'].isin([  
  'PleuralEffusion',
]),:].index
adata2=  adata[cells_of_interest, :]
adata2

View of AnnData object with n_obs × n_vars = 144588 × 22397
    obs: 'ngenes', 'libsize', 'mito_frac', 'RBP_frac', 'batch', 'donor_id', 'treatment', 'procedure', 'histo', 'cell_type_coarse', 'cell_type_fine', 'cell_type_general', 'clusters', 'cell_type_med', 'H_knn', 'development_stage_ontology_term_id', 'sex_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'assay_ontology_term_id', 'is_primary_data', 'tissue_ontology_term_id', 'disease_ontology_term_id', 'organism_ontology_term_id', 'cell_type_ontology_term_id', 'suspension_type', 'HTAN_Biospecimen_ID', 'HTAN_Participant_ID', 'tissue_type', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid'
    var: 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype', 'feature_length', 'feature_type'
    uns: 'citation', 'default_embedding', 'schema_reference', 'schema_version', 'title'
    obsm: 'X_pca', 'X_umap'

In [19]:
cells_of_interest = adata2.obs.loc[~adata2.obs['histo'].isin([  
  'LUAD',
]),:].index
adata3=  adata2[cells_of_interest, :]
adata3

View of AnnData object with n_obs × n_vars = 86909 × 22397
    obs: 'ngenes', 'libsize', 'mito_frac', 'RBP_frac', 'batch', 'donor_id', 'treatment', 'procedure', 'histo', 'cell_type_coarse', 'cell_type_fine', 'cell_type_general', 'clusters', 'cell_type_med', 'H_knn', 'development_stage_ontology_term_id', 'sex_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'assay_ontology_term_id', 'is_primary_data', 'tissue_ontology_term_id', 'disease_ontology_term_id', 'organism_ontology_term_id', 'cell_type_ontology_term_id', 'suspension_type', 'HTAN_Biospecimen_ID', 'HTAN_Participant_ID', 'tissue_type', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid'
    var: 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype', 'feature_length', 'feature_type'
    uns: 'citation', 'default_embedding', 'schema_reference', 'schema_version', 'title'
    obsm: 'X_pca', 'X_umap'

In [ ]:
adata.write('./SCLC_Public_trim_raw.h5ad')